# Lecture 14
## Non Linear Optimization with JuMP (Part 3)
## Date: 29.11

In [1]:
using JuMP, Ipopt, Plots

# Further exercises

## Ex 1

Consider a relationship between a principal and an agent in which the agent's effort influences the result. The agent can choose between low effort $e = 0$, or high effort $e = 3$. His utility function is $U(w,e) = \sqrt{w} - e^2$, where $w$ represent the wage and $e$ the effort. His reservation utility is $21$. The production such that only three results $x$ are possible, where $x$ represents the value of the result to the principal: $x \in \{0,1000,2500\}$. The principal has a utility function $B(x,w) = 10000 + x - w$, where $10000$ is the principal's initial wealth.

The probabilities conditional on effort are the following:

|.       | Pr$(x=0|e)$ | Pr$(x=1000|e)$ | Pr$(x=2500|e)$ |
|:------:|:-----------:|:--------------:|:--------------:|
|$e = 0$ | 0.4         | 0.4            | 0.2            |
|$e = 3$ | 0.2         | 0.4            | 0.4            | 

The timing of the game is as follows. First the risk neutral principal propose one or more contracts. Then the risk averse agent choses the contract, if any, to accept (if does not accept any contract, he will get his reservation utility). Finally the agent chooses the level of effort.

### Point a
Assume agent's effort level is verifiable. Find a sub-game perfect equilibrium of the game that characterize the optimal contract for the principal.

#### Solution Point a

In [2]:
m = Model(with_optimizer(Ipopt.Optimizer));

In [3]:
@variable(m, w[1:3] >= 0);

@variable(m, 0 <= e <= 3);

In [4]:
#Profit Objective
@NLobjective(m, Max, (e/3)*(0.2*(10000+0-w[1])+0.4*(10000+1000-w[2]) + 0.4*(10000+2500-w[3])) + (1-(e/3))*(0.4*(10000+0-w[1])+0.4*(10000+1000-w[2]) + 0.2*(10000+2500-w[3])))

In [5]:
#Participation Constraint
@NLconstraint(m, (e/3)*(0.2*(sqrt(w[1]) - e^2) +0.4*(sqrt(w[2]) - e^2) + 0.4*(sqrt(w[3]) - e^2)) + (1-(e/3))*(0.4*(sqrt(w[1]) - e^2)+0.4*(sqrt(w[2]) - e^2) + 0.2*(sqrt(w[3]) - e^2)) == 21);

In [6]:
JuMP.optimize!(m)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        4
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       20

Total number of variables............................:        4
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        1
                     variables with only upper bounds:        0
Total number of equ

In [7]:
println("** Optimal profits for the firm = ", round(JuMP.objective_value(m)))
println("** Optimal wages given result = ", JuMP.value.(w))
println("** Optimal effort demanded = ", JuMP.value(e))

** Optimal profits for the firm = 10613.0
** Optimal wages given result = [576.104, 576.488, 576.809]
** Optimal effort demanded = 1.7349441509494572


### Point b
Assume now that the only verifiable variable in the relationship is the result $x$.

#### Solution Point b

In [8]:
m = Model(with_optimizer(Ipopt.Optimizer));

In [9]:
@variable(m, w[1:3] >= 0);

eₕ = 3;
eₗ = 0;

In [10]:
#Profit Objective
@NLobjective(m, Max, (0.2*(10000+0-w[1])+0.4*(10000+1000-w[2]) + 0.4*(10000+2500-w[3])))

In [11]:
#Participation Constraint
@NLconstraint(m, (0.2*(sqrt(w[1]) - eₕ^2) +0.4*(sqrt(w[2]) - eₕ^2) + 0.4*(sqrt(w[3]) - eₕ^2)) == 21);

In [12]:
#Incentive Compatibility
@NLconstraint(m, (0.2*(sqrt(w[1]) - eₕ^2) +0.4*(sqrt(w[2]) - eₕ^2) + 0.4*(sqrt(w[3]) - eₕ^2)) >= (0.4*(sqrt(w[1]) - eₗ^2)+0.4*(sqrt(w[2]) - eₗ^2) + 0.2*(sqrt(w[3]) - eₗ^2)));

In [13]:
JuMP.optimize!(m)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        3
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        6

Total number of variables............................:        3
                     variables with only lower bounds:        3
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        1
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [14]:
println("** Optimal profits for the firm = ", round(JuMP.objective_value(m)))
println("** Optimal wages given result = ", round.(JuMP.value.(w)))

** Optimal profits for the firm = 10230.0
** Optimal wages given result = [0.0, 900.0, 2025.0]


### Point c
In a case in which effort is not observable (as in point b) what contract the principal should provide?

## Ex 2

A principal (P) delegates an agent (A) the production of a good. The utility for the principal when the agent produces $q$ units of the good and receives a payment $w$ from the principal is $U_p(q,w) = R(q) - w$, with $R(q) = 14 \ln q$. The utility function for the agent is $U_{AL}(w,q) = w - c_L q$ if he has low costs ($c_L$) and $U_{AH}(w,q) = w - c_H q$ if he has high costs ($c_H$), with $c_H = 4$ and $c_L = 2$. The agent knows whether his cost of production is high or low, and the reservation utility for both types of agent is $0$. The principal can make a take-it-or-leave-it offer to the agent. The quantity produced and the payment to the agent are verifiable. 

### Point a
Assume that the principal knows the production cost of the agent. Find the contracts $(q_L^\star,w_L^\star)$ and $(q_H^\star,q_H^\star)$ the principal would deem optimal to propose to, respectively, an agent with a low cost and one with high cost. Is $q_L^\star$ greater, equal or lower than $q_H^\star$? Is $w_L^\star$ greater, equal or lower than $w_H^\star$?

#### Solution Point a

In [15]:
function R(quantity)
    return 14*log(quantity)
    end;

function Uₚ(quantity,wage)
    return R(quantity) - wage
    end;

function Uₐ(quantity,wage,cost)
    return wage - cost*quantity
    end;

In [16]:
m = Model(with_optimizer(Ipopt.Optimizer));

In [17]:
@variable(m, w[1:2] >= 0);
@variable(m, q[1:2] >= 0);

cₗ = 2;
cₕ = 4;

In [18]:
register(m, :R, 1, R, autodiff=true)
register(m, :Uₚ, 2, Uₚ, autodiff=true)
register(m, :Uₐ, 3, Uₐ, autodiff=true)

In [19]:
#Profit Objective
@NLobjective(m, Max, Uₚ(q[1],w[1]) + Uₚ(q[2],w[2]))

In [20]:
#Participation Constraint
@NLconstraint(m, Uₐ(q[1],w[1],cₗ) + Uₐ(q[2],w[2],cₕ) >= 0);

In [21]:
m

A JuMP Model
Maximization problem with:
Variables: 4
Objective function type: Nonlinear
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
Nonlinear: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.
Names registered in the model: q, w

In [22]:
JuMP.optimize!(m)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        4
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        1
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [23]:
println("Optimal wages are = ", JuMP.value.(w))
println("Optimal quantity = ", JuMP.value.(q))

Optimal wages are = [14.0, 14.0]
Optimal quantity = [7.0, 3.5]


### Point b
Assume now that the principal does not know the production cost of the agent, but she attaches probability 0.25 to the agent having a low cost of production. Find the contracts $(q_L^{\star\star}, w_L^{\star\star})$ and $(q_H^{\star\star}, w_H^{\star\star})$ the principal would deem optimal to propose to the agent, such that an agent with a low cost of production chooses contract $(q_L^{\star\star}, w_L^{\star\star})$ and an agent with high cost of production chooses the contract $(q_H^{\star\star}, w_H^{\star\star})$.

#### Solution Point b

In [24]:
function R(quantity)
    return 14*log(quantity)
    end;

function Uₚ(quantity,wage)
    return R(quantity) - wage
    end;

function Uₐ(quantity,wage,cost)
    return wage - cost*quantity
    end;

In [25]:
m = Model(with_optimizer(Ipopt.Optimizer));

In [26]:
@variable(m, w[1:2] >= 0);
@variable(m, q[1:2] >= 0);

cₗ = 2;
cₕ = 4;

pₗ = 0.25;
pₕ = 1-pₗ;

In [27]:
register(m, :R, 1, R, autodiff=true)
register(m, :Uₚ, 2, Uₚ, autodiff=true)
register(m, :Uₐ, 3, Uₐ, autodiff=true)

In [28]:
#Profit Objective
@NLobjective(m, Max, pₗ*Uₚ(q[1],w[1]) + pₕ*Uₚ(q[2],w[2]))

In [29]:
#Participation Constraint low cost type
@NLconstraint(m, Uₐ(q[1],w[1],cₗ) >= 0);
#Participation Constraint high cost type
@NLconstraint(m, Uₐ(q[2],w[2],cₕ) >= 0);

In [30]:
#Incentive Compatibility low cost type
@NLconstraint(m, Uₐ(q[1],w[1],cₗ) >= Uₐ(q[2],w[2],cₗ));
#Incentive Compatibility high cost type
@NLconstraint(m, Uₐ(q[2],w[2],cₕ) >= Uₐ(q[1],w[1],cₕ));

In [31]:
m

A JuMP Model
Maximization problem with:
Variables: 4
Objective function type: Nonlinear
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 4 constraints
Nonlinear: 4 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: SolverName() attribute not implemented by the optimizer.
Names registered in the model: q, w

In [32]:
JuMP.optimize!(m)

This is Ipopt version 3.12.10, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:       12
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        4
                     variables with only lower bounds:        4
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        4
        inequality constraints with only lower bounds:        4
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 

In [33]:
println(" Optimal wages are = ", round.(JuMP.value.(w)))
println(" Optimal quantity = ", round.(JuMP.value.(q)))

 Optimal wages are = [20.0, 12.0]
 Optimal quantity = [7.0, 3.0]
